In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import os
import sys


term = sys.argv[1] # command line argument for which search results are obtained
# replace with the absolute path to your chromdriver executable
driver = webdriver.Chrome('/Users/namanagarwal/Downloads/instagram_keyterm/chromedriver')

driver.get('https://www.instagram.com/')
sleep(3)

username = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys("namana.neubrain")
password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys("gmpn1634")
log_in = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div')
log_in.click() # login to instagram account
sleep(3)

notnow = driver.find_element_by_xpath('/html/body/div[1]/section/main/div/div/div/div/button')
notnow.click() # disabled notifications in the browser
sleep(3)

no_not = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]')
no_not.click()
sleep(5)

searchbox = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
    (By.XPATH, "//input[@placeholder='Search']")))

searchbox.send_keys(term) # type input term in search bar
sleep(6)

search_results = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
all_search_results_children = search_results.find_elements_by_xpath(".//a")

with open('search_results.txt', 'w') as f:
    for result in all_search_results_children:
        f.write(result.text + '\n\n')
 

ModuleNotFoundError: No module named 'selenium'

In [ ]:
       
file = open('search_results.txt',mode='r')
data = file.read()
data= data.split('\n\n')
data = [row for row in data if row]

os.remove('search_results.txt')

# write results to output.csv file
with open ('output.csv', 'w') as f:
    fw = csv.writer(f)
    fw.writerow(['username', 'description'])
    for row in data:
        details = row.split('\n')
        if len(details) == 2:
            username, description = details
        else:
            username, description = details[0], ''
        fw.writerow([username, description])

driver.close()


In [ ]:
# This code reads instagram id's from the 'input.txt' file and generates 'ouput.csv' file with details : username, description, number of followers, number of users being followed, and number of posts.

import requests
import pandas as pd
import csv
from bs4 import BeautifulSoup

fields = ['username', 'Description', 'followers', 'following', 'posts']
with open('input.txt', 'r') as f, open('output.csv', 'w') as fd:
    wd = csv.writer(fd)
    wd.writerow(fields)
    for username in f:
        username = username.strip()

        html = requests.get('https://www.instagram.com/'+username)
        soup = BeautifulSoup(html.text, 'lxml')
        item = soup.select_one("meta[property='og:description']")

        name = item.find_previous_sibling().get("content").split("•")[0]
        followers = item.get("content").split(",")[0]
        following = item.get("content").split(",")[1].strip()
        posts = item.get('content').split(', ')[2].split('-')[0].strip()
        data = [username, name, followers, following, posts]
        
        print(data, sep='\n')
        wd.writerow(data)
